In [2]:
import torch
import json
import h5py
import numpy as np
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw

/home/skchoi/.conda/envs/sgb_n/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
image_file = json.load(open('/hdd/skchoi/SGG_from_NLS/datasets/vg/image_data.json'))
vocab_file = json.load(open('/hdd/skchoi/SGG_from_NLS/datasets/vg/VG-SGG-dicts-with-attri.json'))
data_file = h5py.File('/hdd/skchoi/SGG_from_NLS/datasets/vg/VG-SGG-with-attri.h5', 'r')
# remove invalid image
corrupted_ims = [1592, 1722, 4616, 4617]
tmp = []
for item in image_file:
    if int(item['image_id']) not in corrupted_ims:
        tmp.append(item)
image_file = tmp

In [20]:
# load detected results
detected_origin_path = '/home/kaihua/checkpoints/vctree-sgcls-only-vis/inference/VG_stanford_filtered_with_attribute_test/'
detected_origin_result = torch.load("/hdd/skchoi/SGG_from_NLS/checkpoints/Language_OpensetCOCO_Uniter/inference/VG_stanford_filtered_with_attribute_test/result_dict.pytorch")
detected_info = json.load(open('/hdd/skchoi/SGG_from_NLS/checkpoints/Language_OpensetCOCO_Uniter/inference/VG_stanford_filtered_with_attribute_test/visual_info.json'))

In [35]:
detected_info[0]['groundtruth']

[[204.1015625, 98.6328125, 217.28515625, 257.8125, 'door'],
 [441.40625, 83.984375, 499.0, 149.4140625, 'house'],
 [403.3203125, 264.16015625, 457.51953125, 279.296875, 'number'],
 [64.453125, 28.3203125, 80.56640625, 201.66015625, 'pole'],
 [133.7890625, 35.64453125, 465.8203125, 349.12109375, 'train'],
 [461.9140625, 127.44140625, 499.0, 179.6875, 'tree'],
 [192.3828125, 122.0703125, 203.61328125, 183.10546875, 'window'],
 [241.2109375, 81.0546875, 450.68359375, 175.29296875, 'windshield'],
 [107.421875, 155.76171875, 125.48828125, 194.82421875, 'woman']]

In [36]:
# get image info by index
def get_info_by_idx(idx, det_input, thres=0.5):
    groundtruth = det_input[idx]['groundtruths']
    prediction =  det_input[idx]['predictions']

    print(groundtruth, prediction)
    '''
    groundtruth = det_input['groundtruths'][idx]
    prediction = det_input['predictions'][idx]
    # image path
    img_path = detected_info[idx]['img_file']
    '''
    # boxes
    boxes = groundtruth.bbox

    # object labels
    idx2label = vocab_file['idx_to_label']
    labels = ['{}-{}'.format(idx,idx2label[str(i)]) for idx, i in enumerate(groundtruth.get_field('labels').tolist())]
    pred_labels = ['{}-{}'.format(idx,idx2label[str(i)]) for idx, i in enumerate(prediction.get_field('pred_labels').tolist())]
    
    # groundtruth relation triplet
    idx2pred = vocab_file['idx_to_predicate']
    gt_rels = groundtruth.get_field('relation_tuple').tolist()
    gt_rels = [(labels[i[0]], idx2pred[str(i[2])], labels[i[1]]) for i in gt_rels]
    # prediction relation triplet
    pred_rel_pair = prediction.get_field('rel_pair_idxs').tolist()
    pred_rel_label = prediction.get_field('pred_rel_scores')
    pred_rel_label[:,0] = 0
    pred_rel_score, pred_rel_label = pred_rel_label.max(-1)
    #mask = pred_rel_score > thres
    #pred_rel_score = pred_rel_score[mask]
    #pred_rel_label = pred_rel_label[mask]
    pred_rels = [(pred_labels[i[0]], idx2pred[str(j)], pred_labels[i[1]]) for i, j in zip(pred_rel_pair, pred_rel_label.tolist())]
    return img_path, boxes, labels, gt_rels, pred_rels, pred_rel_score, pred_rel_label

In [37]:
def draw_single_box(pic, box, color='red', draw_info=None):
    draw = ImageDraw.Draw(pic)
    x1,y1,x2,y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
    draw.rectangle(((x1, y1), (x2, y2)), outline=color)
    if draw_info:
        draw.rectangle(((x1, y1), (x1+50, y1+10)), fill=color)
        info = draw_info
        draw.text((x1, y1), info)

def print_list(name, input_list):
    for i, item in enumerate(input_list):
        print(name + ' ' + str(i) + ': ' + str(item))
    
def draw_image(img_path, boxes, labels, gt_rels, pred_rels, pred_rel_score, pred_rel_label, print_img=True):
    pic = Image.open(img_path)
    num_obj = boxes.shape[0]
    for i in range(num_obj):
        info = labels[i]
        draw_single_box(pic, boxes[i], draw_info=info)
    if print_img:
        display(pic)
    if print_img:
        print('*' * 50)
        print_list('gt_boxes', labels)
        print('*' * 50)
        print_list('gt_rels', gt_rels)
        print('*' * 50)
    print_list('pred_rels', pred_rels[:20])
    print('*' * 50)
    
    return None

In [38]:
def show_selected(idx_list):
    for select_idx in idx_list:
        print(select_idx)
        draw_image(*get_info_by_idx(select_idx, detected_info))
        
# def show_all(start_idx, length):
#     for cand_idx in range(start_idx, start_idx+length):
#         print(cand_idx)
#         draw_image(*get_info_by_idx(cand_idx, detected_origin_result))

In [39]:
show_all(start_idx=0, length=5)
#show_selected([119, 967, 713, 5224, 19681, 25371])

0


KeyError: 0